In [ ]:
from typing import Sequence, Literal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import torch
from torch import Tensor

In [ ]:
plt.rcParams.update({
    'figure.titlesize': 12,
    'axes.titlesize':   10,
    'axes.labelsize':   10,
    'font.size':        8,
    'xtick.labelsize':  8,
    'ytick.labelsize':  8,
    'legend.fontsize':  8,
    'lines.linewidth':  1,
})

COLORS = ['red', 'blue', 'green', 'orange', 'purple',
          'brown', 'pink', 'gray', 'olive', 'cyan',
          'tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:purple',
          'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

<font size="+5">Tree-based Methods</font>

A Decision Tree splits the source dataset into subsets. This splitting process is repeated on each derived subset in a recursive and greedy manner. The recursion is completed when the subset at a node has all the same values of the target variable, or when splitting no longer adds value to the predictions.

<font size="+2">How to split</font>:  
Reducing the variance (for regression) or the impurity (for classification) of the data.

Following the book, this notebook explores the typical tree model: Classification and Regression Tree (CART).

<font size="+5">Regression Trees</font>  

<font size="+3">Problem formulation</font>  

Given data of inputs $\mathbf{X}_{N\times p}$ (numerical or categorical) and labels $\mathbf{Y} \in \mathbb{R}^N$.  
The decision tree $T$ splits our data into $M$ regions $R_1, R_2, \ldots, R_M$.

At each node $m$ corresponding to the region $R_m$, the prediction is modelled as a constant $c_m$ that can represent the node's population.
$$f(x) = \sum_{m=1}^M c_m \mathbf{I}(x\in R_m)$$

Where $c_m$ can be straightforwardly be modelled as the average of $N_m$ samples in the current population at $R_m$.
$$\hat{c}_m = \frac{1}{N_m}\sum_{x_i \in R_m}y_i$$
Note that in this case, $c_m$ also minimizes the squared errors of the predictions $\sum\left(y_i - f(x_i)\right)^2$.

<font size="+3">Splitting criteria</font>  

Consider a split with the $j$-th input, $j \in \{1, 2, \ldots, p\}$, at split point $s \in \mathbb{R}$. That is splitting a region $R$ into two half-planes:
+ $R_\text{L}(j, s) = \{X|X_j \le s\}$
+ $R_\text{R}(j, s) = \{X|X_j > s\}$

We seek to split via *variance reduction*, i.e., finding a split such that:
$$\min_{j, s} \left[\min_{c_\text{L}} \sum_{x_i\in R_\text{L}(j, s)}(y_i - c_\text{L})^2 +
                    \min_{c_\text{R}} \sum_{x_i\in R_\text{R}(j, s)}(y_i - c_\text{R})^2 \right]$$

The inner minimization is acquired when $c_\text{L}, c_\text{R}$ is the average labels, $\forall (j, s)$.
The outer minimization is done by looping through every possible $j$'s and $s$'s.  

Pseudo-code for finding a split at node $m$:
+ **FOR** $j \coloneqq 1$ to $p$:
    + **FOR** $s \coloneqq x_1$ to $x_{N_m}$:
        + **COMPUTE** $X_\text{L}, X_\text{R}$
        + **COMPUTE** $c_\text{L}, c_\text{R}$
        + **COMPUTE** sum of variance $S(j, s) = \sum_{x_i\in R_\text{L}(j, s)}(y_i - c_\text{L})^2 + \sum_{x_i\in R_\text{R}(j, s)}(y_i - c_\text{R})^2$
+ **RETURN** $(j, s)$ with the lowest $S(j, s)$
        

<font size="+5">Classification Trees</font>  

<font size="+3">Problem formulation</font>  

Given data of inputs $\mathbf{X}_{N\times p}$ (numerical or categorical) and labels $\mathbf{Y} \in \{1, 2, \ldots, K\}^N$.  
The decision tree $T$ splits our data into $M$ regions $R_1, R_2, \ldots, R_M$.

At each node $m$ corresponding to the region $R_m$, the prediction is modelled as a probability distribution $\hat{p}_m$ can represent the node's population of $N_m$ samples. For class $k$:
$$\hat{p}_{m, k} = \frac{1}{N_m}\sum_{x_i \in R_m}\mathbf{I}(y_i = k)$$
We can choose either quantity to represent the population and make prediction:
+ Probability distribution: $f(x) = \hat{p}_{m, k}$
+ Major class: $f(x) = \argmax_k \hat{p}_{m, k} = k(m)$ (more popular and lawful)

<font size="+3">Splitting criteria</font>  

Consider a split with the $j$-th input, $j \in \{1, 2, \ldots, p\}$, at split point $s \in \mathbb{R}$. That is splitting a region $R$ into two half-planes:
+ $R_\text{L}(j, s) = \{X|X_j \le s\}$
+ $R_\text{R}(j, s) = \{X|X_j > s\}$

We seek to split via improving the *purity* of the sub-regions, i.e., they are efficently grouped to separate classes.

We can measure the **Impurity** (smaller is better) of each partition $r \in \{R_\text{L}, R_\text{R}\}$ with probability distribution $\hat{p}$ by:
+ Misclassification error: $$Q(r) = \frac{1}{|r|}\sum_{x_i \in r}I(y_i \neq k)$$
+ Gini index: $$Q(r) = \sum_{k=1}^K \sum_{k'\neq k} \hat{p}_{k} \cdot \hat{p}_{k'} = \sum_{k=1}^K \hat{p}_{k} \cdot (1 - \hat{p}_{k})$$
+ Cross-entropy: $$Q(r) = -\sum_{k=1}^K \hat{p}_{k} \log \hat{p}_{k}$$
Then impurity measure of such split can be computed by the weighted average of partitions.
$$\text{GI}(\text{split}) = \frac{|R_\text{L}|}{|R|}\times Q(R_\text{L}) + \frac{|R_\text{R}|}{|R|}\times Q(R_\text{R})$$
We want split that can give purest partitions, i.e., lowest impurity: $\min\left[ \frac{|R_\text{L}|}{|R|}\times Q(R_\text{L}) + \frac{|R_\text{R}|}{|R|}\times Q(R_\text{R}) \right]$

Another splitting criteria is Information Gain:
+ Inspired by Information Entropy: $H(\hat{p}) = -\sum_{k=1}^K \hat{p}_{k} \log_2 \hat{p}_{k}$
+ The purer $\hat{p}$ is, the more predictable its outcomes are, the lower entropy $H(\hat{p})$.
+ We want split that can gain the most information, i.e.: $\max\left[ H(\hat{p}_R) - \left( \frac{|R_\text{L}|}{|R|}\times H(\hat{p}_{R_\text{L}}) + \frac{|R_\text{R}|}{|R|}\times H(\hat{p}_{R_\text{R}}) \right) \right]$

**Example:** Splitting $X = \{\triangle, \triangle, \square, \square, \bigcirc, \bigcirc, \bigcirc\}$ and compute Gini index:
+ Case 1: $X_\text{L} = \{\triangle, \square, \bigcirc\}$ and $X_\text{R} = \{\triangle, \square, \bigcirc, \bigcirc\}$
    + $\text{GI}(X_\text{L}) = \frac{1}{3}\times\frac{2}{3}\times 3 = \frac{2}{3}$
    + $\text{GI}(X_\text{R}) = \frac{1}{4}\times\frac{3}{4}\times 2 + \frac{1}{2}\times\frac{1}{2} = \frac{5}{8}$
    + $\text{GI}(\text{split}) = \frac{3}{7}\times\frac{2}{3} + \frac{4}{7}\times\frac{5}{8} \approx 0.643$
+ Case 2: $X_\text{L} = \{\triangle, \bigcirc, \bigcirc\}$ and $X_\text{R} = \{\triangle, \square, \square, \bigcirc\}$
    + $\text{GI}(X_\text{L}) = \frac{1}{3}\times\frac{2}{3} + \frac{2}{3}\times\frac{1}{3} = \frac{4}{9}$
    + $\text{GI}(X_\text{R}) = \frac{1}{4}\times\frac{3}{4}\times 2 + \frac{1}{2}\times\frac{1}{2} = \frac{5}{8}$
    + $\text{GI}(\text{split}) = \frac{3}{7}\times\frac{4}{9} + \frac{4}{7}\times\frac{5}{8} \approx 0.548$
+ Case 3: $X_\text{L} = \{\triangle, \square\}$ and $X_\text{R} = \{\triangle, \square, \bigcirc, \bigcirc, \bigcirc\}$
    + $\text{GI}(X_\text{L}) = \frac{1}{2}\times\frac{1}{2}\times 2 = \frac{1}{2}$
    + $\text{GI}(X_\text{R}) = \frac{1}{5}\times\frac{4}{5}\times 2 + \frac{3}{5}\times\frac{2}{5} = \frac{14}{25}$
    + $\text{GI}(\text{split}) = \frac{2}{7}\times\frac{1}{2} + \frac{5}{7}\times\frac{14}{25} \approx 0.543$

With the smallest Gini index, case 3 is the best split.

<font size="+5">Regularization</font>  

+ Threshold-based: Stop growing tree if no possible split can reduce variance (for regression) or impurity (for classification) beyond a specified threshold.
+ Complexity-based:
    + Limit the number of nodes
    + Limit the tree depth
    + Cost-complexity to adaptively prune the tree: $C_\alpha = \sum_{m=1}^{|T|} [N_m Q(m)] + \alpha\cdot |T|$
        + $\alpha$: specified regularization parameter
        + $|T|$: number of terminal nodes (leaves)

<font size="+5">Advantages & Disadvantages</font>  

(+):
+ Interpretability, resembles the human-thinking process. Widely used in medicine.
+ Works for both regression and classification tasks.
+ Works for both numerical and categorical data.
+ Minimal data preparation: no need to normalize data or prepare dummy variables (e.g., one-hot encoding)
+ Non-parametric, makes no assumption on the data (but only works well for data separable by constants)
+ In-build feature selection (closer to root = more important)

(-):
+ High variance, overfitting. Because of its greedy and recursive manner, error can propagates from early to late regions.
    + Solution: Bagging (e.g., Random Forest).
+ Greedy: may miss global optimum
+ For data including categorical inputs with different number of classes, Information Gain is biased towards the inputs with more classes 
+ Lack of smoothness, lack of additive structure
    + Solution: Multivariate adaptive regression spline (MARS), trade-off interpretability with performance 

<font size="+5">Other Issues</font>  

+ Categorical Predictors: Large number of classes makes trees more prone to overfitting
+ Loss matrix: The cost of misclassication can be customed to a pre-defined loss matrix. Suitable for case when one outcome is critically avoided (e.g., disease vs. non-disease).
+ Semi-supervised problem (missing predictor values): Two approaches:
    + Categorize samples with missing values as a separate class, therefore finding the pattern of these samples
    + Surrogate predictors
+ Why binary splits? Avoid fragmenting the data too quickly.

In [ ]:
import torch.nn.functional as F

class Node():
    def __init__(self, task:Literal['r', 'c'] = 'c',
                       depth:int = None,
                       path:str = '',
                       parent = None):
        # Arguments
        self.task        = task
        self.depth       = depth
        self.path        = path
        self.parent:Node = parent
        # Pre-allocate
        self.is_leaf = False
        self.info:Tensor = None
        self.feature:Tensor = None
        self.threshold:Tensor = None
        self.children:Sequence[Node] = []
        # Based on task (Classification/Regression)
        self.value:Tensor = None
        if self.task == 'c':
            self.distr:Tensor = None

    def __repr__(self) -> str:
        # Magic attribute to get class name
        if self.depth == 0:
            return f"{self.__class__.__name__}-{self.task} Root, I = {self.info:.3f}, value = {round(self.value, 2)}"
        elif self.depth > 0:
            return f"{self.__class__.__name__}-{self.task} {self.path}, I = {self.info:.3f}, value = {round(self.value, 2)}"

    def branch(self):
        left_node  = Node(task = self.task, depth = self.depth + 1, path = self.path + 'L', parent = self)
        right_node = Node(task = self.task, depth = self.depth + 1, path = self.path + 'R', parent = self)
        return left_node, right_node

class DecisionTree():
    def __init__(self, task:Literal['c', 'r'] = 'c',
                       method_info:Literal['gini', 'entropy', 'var', 'std'] = 'gini',
                       max_depth:int = 4,
                       drop_features:bool = True):
        assert ((task == 'c') & (method_info in ['gini', 'entropy']) |
                (task == 'r') & (method_info in ['var', 'std'])), \
               "Use task 'c' with method_info 'gini' or 'entropy', or task 'r' with method_info 'var' or 'std'"
        self.task          = task
        self.method_info   = method_info
        self.max_depth     = max_depth
        self.drop_features = drop_features

        self.depth:int = 0
        self.feature_picks:Tensor = None

    def __repr__(self) -> str:
        return f"DT-{self.task} with depth {self.depth} (max {self.max_depth}), drop features {'ON' if self.drop_features else 'OFF'}"

    def fit(self, X_train:Tensor, y_train:Tensor):
        self.X_train = X_train
        self.y_train = y_train

        self.num_features = X_train.size()[1]
        self.feature_picks = torch.arange(self.num_features)

        self.root = Node(depth = 0, task = self.task)

        if self.task == 'c':
            self.num_classes = len(y_train.unique())
            self.feed = self.feed_c
            self.compute_info = self.compute_info_c
        elif self.task == 'r':
            self.feed = self.feed_r
            self.compute_info = self.compute_info_r

        idx_root = torch.arange(self.X_train.size()[0])
        self.feed(node = self.root,
                  idx_label = idx_root)
        self.grow(node = self.root,
                  idx_node = idx_root)

    def grow(self, node:Node, idx_node:Tensor):
        max_gain = self.find_best_split(node, idx_node)
        if max_gain > 0:
            # Continue growing
            self.depth = max(self.depth, node.depth + 1)
            left_node, right_node = node.branch()
            node.children = [left_node, right_node]
            # Re-split data by optimal feature and threshold for children nodes
            idx_left, idx_right = self.split(idx_node, feature = node.feature, threshold = node.threshold)
            # Leaf node:
            #  - Has a unique class distribution (contains only one class)
            #  - OR has reached max depth
            for (branch, idx_branch) in zip([left_node, right_node], [idx_left, idx_right]):
                self.feed(branch, idx_branch)
                y_branch = self.y_train[idx_branch]
                if (len(y_branch.unique()) == 1) | (branch.depth == self.max_depth):
                    branch.is_leaf = True
                else:
                    self.grow(branch, idx_branch)
    
    def find_best_split(self, node:Node, idx_node:Tensor) -> Tensor:
        X_node = self.X_train[idx_node]
        # Select random features (sqrt() of previous node's num_features)
        if self.drop_features == True:
            shuffle_idx = torch.randperm(self.feature_picks.numel())
            self.feature_picks = self.feature_picks.view(-1)[shuffle_idx].view(self.feature_picks.size())
            self.feature_picks = self.feature_picks[0:int(self.feature_picks.numel()**0.5)]
        # Split based on the reduced (or not) set of features 
        max_gain = -torch.tensor(float('inf'))
        for feature in torch.arange(self.num_features):
            # thresholds = torch.linspace(start = X_node[:, feature].min(),
            #                             end = X_node[:, feature].max(),
            #                             steps = self.num_splits + 2)[1:self.num_splits+1]
            uniques = X_node[:, feature].sort()[0].unique()
            thresholds = (uniques[1:] + uniques[:-1])/2
            for threshold in thresholds:
                idx_left, idx_right = self.split(idx_node = idx_node,
                                                 feature = feature,
                                                 threshold = threshold)
                gain = self.compute_gain(node,
                                         idx_node = idx_node,
                                         idx_left = idx_left,
                                         idx_right = idx_right)
                if gain > max_gain:
                    max_gain = gain
                    opt_feature = feature
                    opt_threshold = threshold        
        # Update node with optimal split
        node.feature = opt_feature
        node.threshold = opt_threshold
        return max_gain

    def split(self, idx_node:Tensor, feature:Tensor, threshold:Tensor) -> Sequence[Tensor]:
        X_node = self.X_train[idx_node]

        left_ind = X_node[:, feature] <= threshold
        idx_left = idx_node[left_ind]
        idx_right = idx_node[~left_ind]
        return idx_left, idx_right

    def compute_gain(self, node:Node, idx_node:Tensor, idx_left:Tensor, idx_right:Tensor) -> Tensor:
        left_info = self.compute_info(idx_left)
        right_info = self.compute_info(idx_right)
        
        gain = node.info - (idx_left.numel()*left_info + idx_right.numel()*right_info)/idx_node.numel()
        
        # w_node = self.weights[idx_node].sum()
        # w_left = self.weights[idx_left].sum()
        # w_right = self.weights[idx_right].sum()
        # gain = w_node*node.info - (w_left/w_node*left_info + w_right/w_node*right_info)
        return gain

    def feed_c(self, node:Node, idx_label:Tensor):
        label = self.y_train[idx_label]
        node.info = self.compute_info_c(idx_label)
        onehot:Tensor = F.one_hot(label.squeeze(dim = 1), num_classes = self.num_classes)
        
        node.distr = onehot.sum(dim = 0)

        # node.distr = (self.weights[idx_label]*onehot).sum(dim = 0)
        # node.distr = node.distr/node.distr.sum()

        node.value = node.distr.max(dim = 0)[1]

    def feed_r(self, node:Node, idx_label:Tensor):
        label = self.y_train[idx_label]
        node.info = self.compute_info_r(idx_label)
        node.value = label.mean()
    
    def compute_info_c(self, idx_label:Tensor) -> Tensor:
        label = self.y_train[idx_label]
        
        # Classification: reducing Gini impurity or entropy
        onehot:Tensor = F.one_hot(label.squeeze(dim = 1), num_classes = self.num_classes)
        distr = onehot.sum(dim = 0)
        distr = distr/distr.sum()
        if self.method_info == 'gini':
            info = 1 - (distr**2).sum()
        elif self.method_info == 'entropy':
            # Side note:
            # 1. Entropy of a system (all classes) = sum of entropy of its parts (each class)
            # 2. Moreover, if a class has no examples, it is absolutely predictable absent, hence its
            #   entropy is 0.
            # 3. To ease dealing with absent classes (which yields log(0) = -inf), and (2.), the 
            #   computation only considers existing classes.
            info = -(distr[distr != 0]*distr[distr != 0].log()).sum()
        return info
    
    def compute_info_r(self, idx_label:Tensor) -> Tensor:
        label = self.y_train[idx_label]

        # Regression: reducing Variance or Standard deviation
        if self.method_info == 'var':
            info = ((label - label.mean())**2).sum()/label.size()[0]
        elif self.method_info == 'std':
            info = ((label - label.mean())**2).sum().sqrt()/label.size()[0]
        return info

    def print_tree(self):
        def traverse_print(node: Node):
            # Print node
            if node.depth == 0:
                print(f"Root:")
            elif node.depth > 0:
                print(f"{'    '*node.depth}Branch {node.path} " +
                      f"(x{node.parent.feature.item()} {'≤' if node.path[-1] == 'L' else '>'} {node.parent.threshold.item():.2f}):" +
                      f"{f' {node.distr.numpy()}' if self.task == 'c' else ''}" +
                      f"{f' = {round(node.value.item(), 2)}' if node.is_leaf else ''}")
            # Go to children branches
            if node.is_leaf == False:
                for branch in node.children:
                    traverse_print(branch)
            
        traverse_print(self.root)

    def forward(self, input:Tensor, method = 'all') -> Tensor:
        if method == 'each':
            # Method 1: Loop and traverse each example through the tree
            def traverse_forward(node:Node, input:Tensor, yhat) -> Tensor:
                if yhat is not None:
                    return yhat
                elif yhat is None:
                    if node.is_leaf == True:
                        return node.value
                    elif node.is_leaf == False:
                        if input[node.feature] < node.threshold:
                            return traverse_forward(node.children[0], input, yhat)
                        elif input[node.feature] >= node.threshold:
                            return traverse_forward(node.children[1], input, yhat)
            
            yhat = -torch.ones([input.size()[0], 1], dtype={'c':torch.long, 'r':torch.float}.get(self.task))
            for example in torch.arange(input.size()[0]):
                yhat[example] = traverse_forward(self.root, input[example, :], yhat = None)
            return yhat

        elif method == 'all':
            # Method 2: Traverse all examples through the tree at once
            def traverse_forward(node:Node, input:Tensor, yhat:Tensor, yhat_id:Tensor) -> Tensor:
                if node.is_leaf == True:
                    yhat[yhat_id.squeeze()] = node.value
                    return yhat
                elif node.is_leaf == False:
                    left_ind = input[:, node.feature] < node.threshold
                    left_input, right_input = input[left_ind, :], input[~left_ind, :]
                    idx_left, idx_right = yhat_id[left_ind, :], yhat_id[~left_ind, :]
                    for branch, branch_input, idx_branch in zip(node.children, [left_input, right_input], [idx_left, idx_right]):
                        if len(idx_branch) > 0:
                            yhat = traverse_forward(branch, branch_input, yhat, idx_branch)
                    return yhat

            yhat = -torch.ones([input.size()[0], 1], dtype={'c':torch.long, 'r':torch.float}.get(self.task))
            yhat_id = torch.arange(input.size()[0]).unsqueeze(dim = 1)
            yhat = traverse_forward(self.root, input, yhat, yhat_id)
            return yhat

In [ ]:
from utils_data import get_iris

# Task 1: Classification on Iris
def expt_iris(shuffle:bool=True, train_test_split:float=0.5):
    print('Classification Tree on Iris dataset.')
    print(f'Shuffle: {shuffle}')
    print(f'Train-test split: {train_test_split}')
    print()
    # Load Iris
    X, Y = get_iris(shuffle=shuffle)
    num_trains = round(X.size()[0]*train_test_split)
    X_train, y_train = X[0:num_trains], Y[0:num_trains]
    X_test, y_test = X[num_trains:], Y[num_trains:]

    h = DecisionTree(task='c', method_info='gini', max_depth=3, drop_features=True)
    h.fit(X_train, y_train)
    yhat = h.forward(X_test)
    print(h)
    h.print_tree()
    print(f'Accuracy = {((yhat == y_test).sum()/y_test.size()[0]).item():.4f}')

expt_iris(shuffle=True, train_test_split=0.5)

In [ ]:
# Task 2: Regression on 1D Signal
def expt_1d_signal(num_observed:int=20):
    print('Regression Tree on 1D signal.')
    print(f'Number of observed values: {num_observed}')
    # Generate dummy data
    def signal(input):
        return 0.6*input + torch.sin(input)
    X_train = 10*torch.rand([num_observed, 1])
    y_train = signal(X_train)

    h = DecisionTree(task='r', method_info='var', max_depth=3, drop_features=True)
    h.fit(X_train, y_train)

    X_test = torch.arange(start=-2, end=12, step=0.01).unsqueeze(dim = 1)
    y_test = h.forward(X_test)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 6), constrained_layout=True, squeeze=False)
    ax[0, 0].scatter(X_train, y_train,
            color='black', s=40, label='Observed')
    ax[0, 0].step(X_test, y_test,
            linewidth=2, color='blue', label='DT prediction')
    ax[0, 0].plot(X_test, signal(X_test),
            linewidth=1, linestyle='dashed', color='red', alpha=0.5, label='Ground truth')
    ax[0, 0].set(title = h)

    ax[0, 0].legend()
    h.print_tree()
    plt.show()

expt_1d_signal(num_observed=30)

In [ ]:
# Snippet to benchmarking tree.forward() using method = 'each' vs 'all'
# Load Iris
X, Y = get_iris(shuffle=True)
num_trains = round(X.size()[0]*0.5)
X_train, y_train = X[0:num_trains], Y[0:num_trains]
X_test, y_test = X[num_trains:], Y[num_trains:]

h = DecisionTree(task='c', method_info='gini', max_depth=4, drop_features=True)
h.fit(X_train, y_train)
yhat = h.forward(X_test)
print(h)
h.print_tree()
print(f'Accuracy = {((yhat == y_test).sum()/y_test.size()[0]).item():.4f}')


import time

start = time.time()
for i in range(100):
    yhat_each = h.forward(X_test, method='each')
end = time.time()
time1 = end - start
print(f'Forwarding examples one-by-one: {time1:.2e} s')

start = time.time()
for i in range(100):
    yhat_all = h.forward(X_test, method='all')
end = time.time()
time2 = end - start
print(f'Forwarding examples all at once: {time2:.2e} s')

print(f'Faster how many times? {time1/time2:.2f}')
print(f'Confirm results of both methods are the same: {(yhat_each == yhat_all).prod().bool().item()}')